In [28]:
import numpy as np
data_root = './timit_11/'

In [29]:
import numpy as np

print('Loading data ...')

data_root='./timit_11/'
train = np.load(data_root + 'train_11.npy')
train_label = np.load(data_root + 'train_label_11.npy')
test = np.load(data_root + 'test_11.npy')

print('Size of training data: {}'.format(train.shape))
print('Size of testing data: {}'.format(test.shape))

Loading data ...
Size of training data: (1229932, 429)
Size of testing data: (451552, 429)


In [30]:
train = train[:64*10*10,:]
train_label = train_label[:64*10*10]

In [31]:
import torch
from torch.utils.data import Dataset

class TIMITDataset(Dataset):
    def __init__(self, X, y=None):
        self.data = torch.from_numpy(X).float()
        if y is not None:
            y = y.astype(np.int)
            self.label = torch.LongTensor(y)
        else:
            self.label = None

    def __getitem__(self, idx):
        if self.label is not None:
            return self.data[idx], self.label[idx]
        else:
            return self.data[idx]

    def __len__(self):
        return len(self.data)

In [32]:
VAL_RATIO = 0.2

percent = int(train.shape[0] * (1 - VAL_RATIO))
train_x, train_y, val_x, val_y = train[:percent], train_label[:percent], train[percent:], train_label[percent:]
print('Size of training set: {}'.format(train_x.shape))
print('Size of validation set: {}'.format(val_x.shape))

Size of training set: (5120, 429)
Size of validation set: (1280, 429)


In [33]:
BATCH_SIZE = 64

from torch.utils.data import DataLoader

train_set = TIMITDataset(train_x, train_y)
val_set = TIMITDataset(val_x, val_y)
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True) #only shuffle the training data
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False)

In [34]:
import torch
import torch.nn as nn

class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.layer1 = nn.Linear(429, 1024)
        self.layer2 = nn.Linear(1024, 512)
        self.layer3 = nn.Linear(512, 128)
        self.out = nn.Linear(128, 39) 

        self.act_fn = nn.Sigmoid()

    def forward(self, x):
        x = self.layer1(x)
        x = self.act_fn(x)

        x = self.layer2(x)
        x = self.act_fn(x)

        x = self.layer3(x)
        x = self.act_fn(x)

        x = self.out(x)
        
        return x

In [35]:
def get_device():
      return 'cuda' if torch.cuda.is_available() else 'cpu'

In [36]:
def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  
    np.random.seed(seed)  
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

In [37]:
same_seeds(0)

# get device 
device = get_device()
print(f'DEVICE: {device}')

# training parameters
num_epoch = 20               # number of training epoch
learning_rate = 0.0001       # learning rate

# the path where checkpoint saved
model_path = './model.ckpt'

# create model, define a loss function, and optimizer
model = Classifier().to(device)
criterion = nn.CrossEntropyLoss() 
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

DEVICE: cuda


In [38]:
best_acc = 0.0
from tqdm import tqdm
for epoch in tqdm(range(num_epoch)):
    train_acc = 0.0
    train_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0

    # training
    model.train() # set the model to training mode
    for i, data in enumerate(train_loader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad() 
        outputs = model(inputs) 
       
        batch_loss = criterion(outputs, labels)
        _, train_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
        batch_loss.backward() 
        optimizer.step() 

        train_acc += (train_pred.cpu() == labels.cpu()).sum().item()
        train_loss += batch_loss.item()

    # validation
    if len(val_set) > 0:
        model.eval() # set the model to evaluation mode
        with torch.no_grad():
            for i, data in enumerate(val_loader):
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                batch_loss = criterion(outputs, labels) 
                _, val_pred = torch.max(outputs, 1) 
            
                val_acc += (val_pred.cpu() == labels.cpu()).sum().item() # get the index of the class with the highest probability
                val_loss += batch_loss.item()

            print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f} | Val Acc: {:3.6f} loss: {:3.6f}'.format(
                epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader), val_acc/len(val_set), val_loss/len(val_loader)
            ))

            # if the model improves, save a checkpoint at this epoch
            if val_acc > best_acc:
                best_acc = val_acc
                torch.save(model.state_dict(), model_path)
                print('saving model with acc {:.3f}'.format(best_acc/len(val_set)))
    else:
        print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f}'.format(
            epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader)
        ))

# if not validating, save the last epoch
if len(val_set) == 0:
    torch.save(model.state_dict(), model_path)
    print('saving model at last epoch')

  5%|████▏                                                                              | 1/20 [00:00<00:09,  2.08it/s]

[001/020] Train Acc: 0.111133 Loss: 3.447898 | Val Acc: 0.144531 loss: 3.296363
saving model with acc 0.145


 10%|████████▎                                                                          | 2/20 [00:00<00:08,  2.23it/s]

[002/020] Train Acc: 0.141406 Loss: 3.281695 | Val Acc: 0.144531 loss: 3.246792


 15%|████████████▍                                                                      | 3/20 [00:01<00:07,  2.24it/s]

...

[018/020] Train Acc: 0.408398 Loss: 2.112412 | Val Acc: 0.387500 loss: 2.384258
saving model with acc 0.388


 95%|█████████████████████████████████████████████████████████████████████████████▉    | 19/20 [00:08<00:00,  2.42it/s]

[019/020] Train Acc: 0.414258 Loss: 2.081889 | Val Acc: 0.382812 loss: 2.373496


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:08<00:00,  2.31it/s]

[020/020] Train Acc: 0.415820 Loss: 2.050104 | Val Acc: 0.389062 loss: 2.345814
saving model with acc 0.389


In [39]:
X, y = next(iter(train_loader))

In [40]:
X.shape

torch.Size([64, 429])

In [41]:
y.shape

torch.Size([64])

In [42]:
X= X.to('cuda')
y=y.to('cuda')

In [43]:
output = model(X)

In [44]:
output

tensor([[-1.4137e+00, -7.8486e-01, -1.0344e+00,  ...,  6.4967e-01,
          3.1024e-01,  4.5293e+00],
        [ 2.8195e-02,  2.0058e+00,  2.4369e+00,  ..., -2.0895e+00,
         -1.2989e+00, -8.9782e-01],
        [ 1.0262e-01,  1.7809e+00,  2.2313e+00,  ..., -1.8654e+00,
         -1.0747e+00,  2.0184e-01],
        ...,
        [-1.2509e+00, -9.3875e-01, -1.2110e+00,  ...,  5.5823e-01,
          3.3478e-01,  4.6001e+00],
        [-1.8139e+00, -5.4832e-01, -3.4450e-03,  ...,  4.8008e-01,
          2.5284e-01,  1.8269e+00],
        [ 1.2330e+00,  2.8875e+00,  2.3593e+00,  ..., -2.7334e+00,
         -1.4462e+00, -2.9041e-01]], device='cuda:0', grad_fn=<AddmmBackward>)

In [45]:
output.shape

torch.Size([64, 39])

In [46]:
criterion(X,y)

tensor(6.6600, device='cuda:0')